In [1]:
import torch

### top-k策略

#### Top-k 采样中，可以将采样的词元限制在前 k 个最可能的词元上，并通过掩码概率分数的方式来排除其他词元

#### Top-k 方法用负无穷值（-inf）替换所有未选择的 logits，因此在计算 softmax 值时，非前 k词元的概率分数为 0，剩余的概率总和为 1。

In [ ]:
def generate(mdodel, idx, max_new_tokens, context_size, temputure, 
            top_k=None, eos_id=None):
    for _ in range(max_new_tokens):
        idx = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx)
        logits = logits[:, -1, :]
        if top_k is not None:
            top_logits, _  = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(
                logits < min_val,
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )
        if temputure > 0.0:
            logits = logits / temputure
            prob = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(prob, num_samples=1)
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)
        if idx_next == eos_id:
            break
        idx = torch.cat(idx, idx_next)
    return idx